In [2]:
import os
import re
import nltk

import spacy
# spacy.cli.download("nl_core_news_lg")
nlp = spacy.load("nl_core_news_lg") 

import pandas as pd
import numpy as np

import textdescriptives as td

import folia.main as folia #https://foliapy.readthedocs.io/en/latest/folia.html#loading-a-document
from pathlib import Path
from collections import defaultdict
import pandas as pd
from tqdm.notebook import tqdm

BSdir = Path('/Users/sabijn/Documents/PhD/Datasets/BS_only_narrative')

## Create Basiscript lexicon

In [ ]:
BSfile = Path(os.getcwd()) / Path('datasets/BSlexicon.csv')

if BSfile.exists():
    df_lexicon = pd.read_csv(BSfile)
else:
    BSlexicon = defaultdict(int)
    for file in tqdm(BSdir.glob('*.xml')):
        doc = folia.Document(file=str(file))
        enriched_doc = nlp(doc.text())
        for token in enriched_doc:
            BSlexicon[(token.lemma_, token.pos_)] += 1

    df_lexicon = pd.DataFrame([(k[0], k[1], v) for k, v in BSlexicon.items()],
        columns=['lemma', 'pos', 'freq'])

    df_lexicon.to_csv('datasets/BSlexicon.csv')

## Create Basiscript bigram lexicon

In [4]:
BSfile = Path('/Users/sabijn/Documents/PhD/code/storylm_p1_data/datasets/BS_bigram_lexicon_part.csv')

if BSfile.exists():
    df_bi_lexicon = pd.read_csv(BSfile)
else:
    BS_bi_lexicon = defaultdict(int)
    for file in tqdm(BSdir.glob('*.xml')):
        doc = folia.Document(file=str(file))
        enriched_doc = nlp(doc.text())

        for i in range(len(enriched_doc) - 1):
            BS_bi_lexicon[((enriched_doc[i].lemma_, enriched_doc[i + 1].lemma_), (enriched_doc[i].pos_, enriched_doc[i + 1].pos_))] += 1
        
        if i == 100_000:
            break

    df_bi_lexicon = pd.DataFrame([(k[0], k[1], v) for k, v in BS_bi_lexicon.items()],
        columns=['lemma', 'pos', 'freq'])

    df_bi_lexicon.to_csv(BSfile)

0it [00:00, ?it/s]

## Create dependency-constrained lexicon

In [2]:
def select_dep_pairs(node, selected):
    temp = {child.dep_ : child for child in list(node.children)}

    if node.dep_ in ['ROOT', 'ccomp', 'xcomp', 'conj']:
        for (child_dep, child) in temp.items():
            if child_dep in ['nsubj', 'obj', 'cop', 'nsubj:pass'] and child.pos_ != 'PRON' and child.text != 'einde':
                selected[((child.lemma_, node.lemma_), (child_dep, node.dep_))] += 1
    
    if node.pos_ == 'NOUN':
        for child in list(node.children):
            if child.pos_ == 'ADJ' and child.dep_ == 'amod':
                selected[((child.lemma_, node.lemma_), (child.dep_, node.dep_))] += 1

    if node.n_lefts + node.n_rights > 0:
        for child in node.children:
            select_dep_pairs(child, selected=selected)

    return selected

In [ ]:
BSfile = Path('/Users/sabijn/Documents/PhD/code/storylm_p1_data/datasets/BSdeplexicon_part.csv')


if BSfile.exists():
    df_dep_lexicon = pd.read_csv(BSfile)
else:
    BS_dep_lexicon = defaultdict(int)
    for i, file in enumerate(BSdir.glob('*.xml')):
        doc = nlp(folia.Document(file=str(file)).text())

        for sent in doc.sents:
            pair_dict = select_dep_pairs(sent.root, BS_dep_lexicon)

        if i == 100_000:
            break

    df_dp_lexicon = pd.DataFrame([(k[0], k[1], v) for k, v in BS_dep_lexicon.items()],
        columns=['lemma', 'pos', 'freq'])

    df_dp_lexicon.to_csv(BSfile)

OSError: Cannot save file into a non-existent directory: '/Users/sabijn/Documents/PhD/code/storylm_p1_data/notebooks/datasets'

## Transform dep to no dep lexicon

In [ ]:
import ast 

new_lexicon_file = Path('/Users/sabijn/Documents/PhD/code/storylm_p1_data/datasets/BSdep_without_deps_part.csv')
old_lexicon_file = Path('/Users/sabijn/Documents/PhD/code/storylm_p1_data/datasets/BSdeplexicon_part.csv')
df_old = pd.read_csv(old_lexicon_file)

df_stripped = df_old.drop(columns='pos')
df_grouped = df_old.groupby('lemma', as_index=False)['freq'].sum()

# Convert string to tuple
df_grouped['lemma'] = df_grouped['lemma'].apply(ast.literal_eval)

# Split the tuple into two separate columns
df_grouped[['lemma1', 'lemma2']] = pd.DataFrame(df_grouped['lemma'].tolist(), index=df_grouped.index)

# Drop the original tuple column
df_grouped = df_grouped.drop(columns='lemma')

df_grouped.to_csv(new_lexicon_file)

In [7]:
import matplotlib.pyplot as plt
from collections import Counter

Counter(df_grouped['freq'])

Counter({1: 142935,
         2: 23410,
         3: 9001,
         4: 5094,
         5: 3075,
         6: 2140,
         7: 1568,
         8: 1212,
         9: 908,
         10: 800,
         11: 641,
         12: 560,
         13: 460,
         14: 427,
         16: 320,
         15: 319,
         18: 274,
         17: 262,
         20: 228,
         19: 216,
         22: 181,
         21: 178,
         23: 173,
         24: 158,
         25: 141,
         26: 132,
         27: 116,
         29: 112,
         32: 96,
         30: 95,
         31: 91,
         34: 90,
         28: 86,
         33: 74,
         36: 69,
         35: 68,
         37: 63,
         38: 58,
         42: 57,
         40: 54,
         46: 46,
         44: 46,
         39: 46,
         41: 45,
         43: 44,
         47: 44,
         52: 38,
         56: 37,
         50: 36,
         57: 35,
         53: 34,
         49: 34,
         48: 33,
         59: 32,
         45: 31,
         51: 31,
         54: 30,
 

In [ ]:
BSfile_unigram = Path('/Users/sabijn/Documents/PhD/code/storylm_p1_data/datasets/BS_unigrams_part.csv')

if BSfile_unigram.exists():
    df_lexicon = pd.read_csv(BSfile_unigram)
else:
    BSlexicon = defaultdict(int)
    for file in tqdm(BSdir.glob('*.xml')):
        doc = folia.Document(file=str(file))
        enriched_doc = nlp(doc.text())
        for token in enriched_doc:
            BSlexicon[token.pos_] += 1

        if i == 100_000:
            break

    df_lexicon = pd.DataFrame([(k[0], k[1], v) for k, v in BSlexicon.items()],
        columns=['pos', 'freq'])

    df_lexicon.to_csv(BSfile_unigram)